In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import recall_score, make_scorer, accuracy_score, precision_score,classification_report, confusion_matrix, roc_curve, roc_auc_score,auc
from pandas import set_option


In [2]:
set_option('display.max_row', 10)
set_option('display.width', 100)
set_option('precision', 2)
set_option('display.max_column', 40)

In [3]:
df= pd.read_csv('CC.csv') 
df_korban_fraud = df_korban[df_korban.LABEL==1].copy()
df_non_korban = pd.read_csv('datanonfraud.csv')
frames= [df_korban_fraud, df_non_korban.sample(8821)] # SAMPEL TERDIRI DARI 10000 BARIS YANG DIAMBIL SECARA ACAK
data= pd.concat(frames)
y_true = data['LABEL']
data.head()
y_true=data['LABEL']

C:\Users\leno\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data[data['LABEL']==1].describe()    # Describing the data

,LABEL,fiturid,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
count,1179.0,1.18e+03,1.18e+03,1.18e+03,1179.00,1179.00,1179.00,1179.00,1.18e+03,1.18e+03,1.18e+03,1.18e+03
mean,1.0,1.13e+08,2.23e+07,2.44e+06,42452.84,3.21,0.49,64.25,3.45e+04,7.54e+05,3.45e+04,4.84e+01
std,0.0,3.04e+07,1.89e+08,9.12e+06,20143.72,3.27,0.50,626.61,6.83e+05,1.56e+07,6.83e+05,1.49e+03
min,1.0,1.03e+08,0.00e+00,1.00e+03,8.00,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,4.08e-05
25%,1.0,1.03e+08,0.00e+00,4.00e+05,27799.00,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,9.62e-02
50%,1.0,1.03e+08,7.23e+05,1.25e+06,41328.00,2.00,0.00,25.00,6.13e-01,9.09e-01,3.33e-01,3.84e-01
75%,1.0,1.03e+08,6.56e+06,2.00e+06,59873.00,4.00,1.00,34.00,2.04e+00,2.53e+00,9.50e-01,9.04e-01
max,1.0,2.06e+08,2.87e+09,2.65e+08,85168.00,32.00,1.00,16804.00,1.50e+07,5.23e+08,1.50e+07,5.10e+04


In [5]:
data.shape

(10000, 19)

In [6]:
data.head()

,LABEL,fiturid,responcode,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
3853,1,103010020,0,1.15e+06,100000,28893,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,1,0,0.0,0.0,0.86,0.08
8023,1,103010020,51,0.00e+00,200000,4099,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.0,0.0,0.60,3.83
8024,1,103010020,0,5.23e+04,150000,4123,ATM_BRI,BRITAMA,TarikTunai,True,dalam,51,2,0,24,0.0,0.0,0.25,6.70
9960,1,103010020,0,5.00e+05,1000000,48153,ATM_BRI,SIMPEDES,TarikTunai,True,dalam,0,1,0,0,0.0,0.0,0.33,2.00
10485,1,103010020,0,4.59e+07,1000000,69395,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.0,0.0,0.60,0.02


In [7]:
data.duplicated().sum()

3

In [8]:
data.loc[data.duplicated(keep = False), :] 

,LABEL,fiturid,responcode,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
270212,0,201010170,0,1.20e+07,0,55457,ATM MERAH PUTIH,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.00,0.0
270596,0,201010170,0,5.47e+05,0,74871,ATM MERAH PUTIH,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.00,0.0
78942,0,103010020,0,2.56e+06,200000,23513,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,2,0,14,0.0,0.0,0.35,0.2
270239,0,201010170,0,1.20e+07,0,55457,ATM MERAH PUTIH,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.00,0.0
270617,0,201010170,0,5.47e+05,0,74871,ATM MERAH PUTIH,BRITAMA,CekSaldo,False,dalam,0,0,1,0,0.0,0.0,0.00,0.0
79705,0,103010020,0,2.56e+06,200000,23513,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,2,0,14,0.0,0.0,0.35,0.2


In [9]:
data.drop_duplicates(keep= 'first').shape

(9997, 19)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 3853 to 13362
Data columns (total 19 columns):
LABEL                10000 non-null int64
fiturid              10000 non-null int64
responcode           10000 non-null object
avlblnc              10000 non-null float64
txamount             10000 non-null int64
transtime            10000 non-null int64
terminaltype         10000 non-null object
jenis_kartu          9999 non-null object
groupname            10000 non-null object
dana_keluar          10000 non-null bool
lokasi               10000 non-null object
responcode_before    9999 non-null object
seq.trxday           10000 non-null int64
CekSaldo.before      10000 non-null int64
diff.time            10000 non-null int64
diff.txamount        10000 non-null float64
diff.txamount.trx    10000 non-null float64
diff.txamount.bef    10000 non-null float64
percent_saldo        10000 non-null float64
dtypes: bool(1), float64(5), int64(7), object(6)
memory usage: 1.5+ MB


In [11]:
data.responcode =pd.to_numeric(data.responcode, errors='coerce').fillna(0, downcast='infer') # TERDAPAT NAN YANG HARUS DIHILANGKAN
data.responcode_before =pd.to_numeric(data.responcode_before, errors='coerce').fillna(0, downcast='infer')

In [12]:
data[data['LABEL']==1].describe()  

,LABEL,fiturid,responcode,avlblnc,txamount,transtime,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo
count,1179.0,1.18e+03,1179.00,1.18e+03,1.18e+03,1179.00,1179.0,1179.00,1179.00,1179.00,1.18e+03,1.18e+03,1.18e+03,1.18e+03
mean,1.0,1.13e+08,14.80,2.23e+07,2.44e+06,42452.84,12.0,3.21,0.49,64.25,3.45e+04,7.54e+05,3.45e+04,4.84e+01
std,0.0,3.04e+07,25.06,1.89e+08,9.12e+06,20143.72,23.2,3.27,0.50,626.61,6.83e+05,1.56e+07,6.83e+05,1.49e+03
min,1.0,1.03e+08,0.00,0.00e+00,1.00e+03,8.00,0.0,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,4.08e-05
25%,1.0,1.03e+08,0.00,0.00e+00,4.00e+05,27799.00,0.0,1.00,0.00,0.00,0.00e+00,0.00e+00,0.00e+00,9.62e-02
50%,1.0,1.03e+08,0.00,7.23e+05,1.25e+06,41328.00,0.0,2.00,0.00,25.00,6.13e-01,9.09e-01,3.33e-01,3.84e-01
75%,1.0,1.03e+08,51.00,6.56e+06,2.00e+06,59873.00,0.0,4.00,1.00,34.00,2.04e+00,2.53e+00,9.50e-01,9.04e-01
max,1.0,2.06e+08,93.00,2.87e+09,2.65e+08,85168.00,61.0,32.00,1.00,16804.00,1.50e+07,5.23e+08,1.50e+07,5.10e+04


In [13]:
data= data.dropna()

In [14]:
one_hot = pd.get_dummies(data['responcode']) # RESPONCODE MEMILIKI TYPE OBJECT SEHINGGA HARUS DILAKUKAN ONE HOT ENCODING 
data= data.drop('responcode', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,responcode_before,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,93,96
3853,1,103010020,1.15e+06,100000,28893,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,0
8023,1,103010020,0.00e+00,200000,4099,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.00,0.00,0.60,3.83,0,0,1,0,0,0,0
8024,1,103010020,5.23e+04,150000,4123,ATM_BRI,BRITAMA,TarikTunai,True,dalam,51,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,0
9960,1,103010020,5.00e+05,1000000,48153,ATM_BRI,SIMPEDES,TarikTunai,True,dalam,0,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,0
10485,1,103010020,4.59e+07,1000000,69395,ATM_BRI,BRITAMA,TarikTunai,True,dalam,0,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246513,0,205010360,0.00e+00,150000,59695,INTERNET BANKING,BRITAMA,TransferBRI,True,dalam,0,30,1,865,23.99,23.99,0.71,1.03,1,0,0,0,0,0,0
180984,0,205010360,0.00e+00,475000,56168,INTERNET BANKING,BRITAMA,TransferBRI,True,dalam,0,148,1,30,105.86,105.86,3.75,1.30,1,0,0,0,0,0,0
151120,0,205010360,0.00e+00,61000,80868,INTERNET BANKING,BRITAMA,TransferBRI,True,dalam,0,205,1,30,211.53,211.53,0.76,2.36,1,0,0,0,0,0,0
165297,0,205010360,0.00e+00,200000,51674,INTERNET BANKING,BRITAMA,TransferBRI,True,dalam,0,84,1,36,34.46,34.46,0.56,2.09,1,0,0,0,0,0,0


In [15]:
one_hot = pd.get_dummies(data['responcode_before'])# RESPONCODE_BEFORE MEMILIKI TYPE OBJECT SEHINGGA HARUS DILAKUKAN ONE HOT ENCODING 
data= data.drop('responcode_before', axis =1)
data= pd.concat([data, one_hot], axis=1)


In [16]:
data.head()

,LABEL,fiturid,avlblnc,txamount,transtime,terminaltype,jenis_kartu,groupname,dana_keluar,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,93,96,0,3,30,51,53,61,92
3853,1,103010020,1.15e+06,100000,28893,ATM_BRI,BRITAMA,TarikTunai,True,dalam,1,1,0,0.0,0.0,0.86,0.08,1,0,0,0,0,0,0,1,0,0,0,0,0,0
8023,1,103010020,0.00e+00,200000,4099,ATM_BRI,BRITAMA,TarikTunai,True,dalam,1,0,0,0.0,0.0,0.60,3.83,0,0,1,0,0,0,0,1,0,0,0,0,0,0
8024,1,103010020,5.23e+04,150000,4123,ATM_BRI,BRITAMA,TarikTunai,True,dalam,2,0,24,0.0,0.0,0.25,6.70,1,0,0,0,0,0,0,0,0,0,1,0,0,0
9960,1,103010020,5.00e+05,1000000,48153,ATM_BRI,SIMPEDES,TarikTunai,True,dalam,1,0,0,0.0,0.0,0.33,2.00,1,0,0,0,0,0,0,1,0,0,0,0,0,0
10485,1,103010020,4.59e+07,1000000,69395,ATM_BRI,BRITAMA,TarikTunai,True,dalam,1,0,0,0.0,0.0,0.60,0.02,1,0,0,0,0,0,0,1,0,0,0,0,0,0


In [17]:
one_hot = pd.get_dummies(data['terminaltype'])
data= data.drop('terminaltype', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,jenis_kartu,groupname,dana_keluar,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,93,96,0,3,30,51,53,61,92,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING
3853,1,103010020,1.15e+06,100000,28893,BRITAMA,TarikTunai,True,dalam,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8023,1,103010020,0.00e+00,200000,4099,BRITAMA,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.60,3.83,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8024,1,103010020,5.23e+04,150000,4123,BRITAMA,TarikTunai,True,dalam,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
9960,1,103010020,5.00e+05,1000000,48153,SIMPEDES,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
10485,1,103010020,4.59e+07,1000000,69395,BRITAMA,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246513,0,205010360,0.00e+00,150000,59695,BRITAMA,TransferBRI,True,dalam,30,1,865,23.99,23.99,0.71,1.03,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
180984,0,205010360,0.00e+00,475000,56168,BRITAMA,TransferBRI,True,dalam,148,1,30,105.86,105.86,3.75,1.30,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
151120,0,205010360,0.00e+00,61000,80868,BRITAMA,TransferBRI,True,dalam,205,1,30,211.53,211.53,0.76,2.36,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
165297,0,205010360,0.00e+00,200000,51674,BRITAMA,TransferBRI,True,dalam,84,1,36,34.46,34.46,0.56,2.09,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [18]:
one_hot = pd.get_dummies(data['jenis_kartu'])
data= data.drop('jenis_kartu', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,groupname,dana_keluar,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,...,0,3,30,51,53,61,92,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES
3853,1,103010020,1.15e+06,100000,28893,TarikTunai,True,dalam,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
8023,1,103010020,0.00e+00,200000,4099,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.60,3.83,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
8024,1,103010020,5.23e+04,150000,4123,TarikTunai,True,dalam,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
9960,1,103010020,5.00e+05,1000000,48153,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
10485,1,103010020,4.59e+07,1000000,69395,TarikTunai,True,dalam,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246513,0,205010360,0.00e+00,150000,59695,TransferBRI,True,dalam,30,1,865,23.99,23.99,0.71,1.03,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
180984,0,205010360,0.00e+00,475000,56168,TransferBRI,True,dalam,148,1,30,105.86,105.86,3.75,1.30,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
151120,0,205010360,0.00e+00,61000,80868,TransferBRI,True,dalam,205,1,30,211.53,211.53,0.76,2.36,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
165297,0,205010360,0.00e+00,200000,51674,TransferBRI,True,dalam,84,1,36,34.46,34.46,0.56,2.09,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [19]:
one_hot = pd.get_dummies(data['groupname'])
data= data.drop('groupname', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,dana_keluar,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,93,...,61,92,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama
3853,1,103010020,1.15e+06,100000,28893,True,dalam,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
8023,1,103010020,0.00e+00,200000,4099,True,dalam,1,0,0,0.00,0.00,0.60,3.83,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
8024,1,103010020,5.23e+04,150000,4123,True,dalam,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
9960,1,103010020,5.00e+05,1000000,48153,True,dalam,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
10485,1,103010020,4.59e+07,1000000,69395,True,dalam,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246513,0,205010360,0.00e+00,150000,59695,True,dalam,30,1,865,23.99,23.99,0.71,1.03,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
180984,0,205010360,0.00e+00,475000,56168,True,dalam,148,1,30,105.86,105.86,3.75,1.30,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
151120,0,205010360,0.00e+00,61000,80868,True,dalam,205,1,30,211.53,211.53,0.76,2.36,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
165297,0,205010360,0.00e+00,200000,51674,True,dalam,84,1,36,34.46,34.46,0.56,2.09,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0


In [20]:
one_hot = pd.get_dummies(data['dana_keluar'])
data= data.drop('dana_keluar', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,lokasi,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,93,96,...,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True
3853,1,103010020,1.15e+06,100000,28893,dalam,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8023,1,103010020,0.00e+00,200000,4099,dalam,1,0,0,0.00,0.00,0.60,3.83,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8024,1,103010020,5.23e+04,150000,4123,dalam,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
9960,1,103010020,5.00e+05,1000000,48153,dalam,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
10485,1,103010020,4.59e+07,1000000,69395,dalam,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246513,0,205010360,0.00e+00,150000,59695,dalam,30,1,865,23.99,23.99,0.71,1.03,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1
180984,0,205010360,0.00e+00,475000,56168,dalam,148,1,30,105.86,105.86,3.75,1.30,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1
151120,0,205010360,0.00e+00,61000,80868,dalam,205,1,30,211.53,211.53,0.76,2.36,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1
165297,0,205010360,0.00e+00,200000,51674,dalam,84,1,36,34.46,34.46,0.56,2.09,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1


In [21]:
one_hot = pd.get_dummies(data['lokasi'])
data= data.drop('lokasi', axis =1)
data= pd.concat([data, one_hot], axis=1)
data

,LABEL,fiturid,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,93,96,0,...,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True,dalam,luar
3853,1,103010020,1.15e+06,100000,28893,1,1,0,0.00,0.00,0.86,0.08,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8023,1,103010020,0.00e+00,200000,4099,1,0,0,0.00,0.00,0.60,3.83,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8024,1,103010020,5.23e+04,150000,4123,2,0,24,0.00,0.00,0.25,6.70,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
9960,1,103010020,5.00e+05,1000000,48153,1,0,0,0.00,0.00,0.33,2.00,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0
10485,1,103010020,4.59e+07,1000000,69395,1,0,0,0.00,0.00,0.60,0.02,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246513,0,205010360,0.00e+00,150000,59695,30,1,865,23.99,23.99,0.71,1.03,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0
180984,0,205010360,0.00e+00,475000,56168,148,1,30,105.86,105.86,3.75,1.30,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0
151120,0,205010360,0.00e+00,61000,80868,205,1,30,211.53,211.53,0.76,2.36,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0
165297,0,205010360,0.00e+00,200000,51674,84,1,36,34.46,34.46,0.56,2.09,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0


In [22]:
X = data.iloc[:, data.columns != "LABEL"]
Y = data.iloc[:, data.columns == "LABEL"]

In [23]:
X.head()


,fiturid,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,93,96,0,3,...,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True,dalam,luar
3853,103010020,1.15e+06,100000,28893,1,1,0,0.0,0.0,0.86,0.08,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8023,103010020,0.00e+00,200000,4099,1,0,0,0.0,0.0,0.60,3.83,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
8024,103010020,5.23e+04,150000,4123,2,0,24,0.0,0.0,0.25,6.70,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
9960,103010020,5.00e+05,1000000,48153,1,0,0,0.0,0.0,0.33,2.00,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0
10485,103010020,4.59e+07,1000000,69395,1,0,0,0.0,0.0,0.60,0.02,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0


In [24]:
from sklearn.model_selection import train_test_split,GridSearchCV, StratifiedKFold
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.37, random_state= None) # UKURAN DATA TESTING YAKNI 2000 BARIS

In [25]:
# STANDARDISASI NILAI PADA SETIAP FITUR
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [26]:
print(X_train_std)

[[ 0.38054977 -0.02511548 -0.20564349 ... -1.42012762  0.1675484
  -0.1675484 ]
 [ 0.48755534 -0.12486777 -0.19585925 ...  0.70416207  0.1675484
  -0.1675484 ]
 [-2.24096599 -0.12486777  0.18572581 ...  0.70416207  0.1675484
  -0.1675484 ]
 ...
 [ 0.48755534 -0.12486777 -0.2054478  ...  0.70416207  0.1675484
  -0.1675484 ]
 [ 0.38054977 -0.10244236 -0.20564349 ... -1.42012762  0.1675484
  -0.1675484 ]
 [ 0.38054977 -0.12469669 -0.20564349 ... -1.42012762  0.1675484
  -0.1675484 ]]


In [27]:
# FITUR ID SUDAH DIWALIKI OLEH GROUPNAME
data.drop(['fiturid', 'dalam', 'luar', 'false', 'true'], axis=1, inplace=True)
data.head()

,LABEL,avlblnc,txamount,transtime,seq.trxday,CekSaldo.before,diff.time,diff.txamount,diff.txamount.trx,diff.txamount.bef,percent_saldo,0,30,51,53,61,93,96,0,3,...,ATM MERAH PUTIH,ATM_BRI,Bersama,CDM,EDC,Himbara,INTERNET BANKING,Maestro,Prima,SMS BANKING,BRITAMA,PRIORITAS,SIMPEDES,CekSaldo,TarikTunai,TransferBRI,TransferBRIVA,TransferBersama,False,True
3853,1,1.15e+06,100000,28893,1,1,0,0.0,0.0,0.86,0.08,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8023,1,0.00e+00,200000,4099,1,0,0,0.0,0.0,0.60,3.83,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
8024,1,5.23e+04,150000,4123,2,0,24,0.0,0.0,0.25,6.70,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
9960,1,5.00e+05,1000000,48153,1,0,0,0.0,0.0,0.33,2.00,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
10485,1,4.59e+07,1000000,69395,1,0,0,0.0,0.0,0.60,0.02,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1


In [28]:
# Optimisasi dengan Hyperparameter Tuning
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_grid = [{'C': param_range,
'kernel': ['linear']},
{'C': param_range,
'gamma': param_range,
'kernel': ['rbf']}]

clf = SVC()

scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score)
}


In [29]:
def grid_search_wrapper(refit_score='recall_score'):
    """
    fits a GridSearchCV classifier using refit_score for optimization
    prints classifier performance metrics
    """
    skf = StratifiedKFold(n_splits=5)
    grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score,
                           cv=skf, return_train_score=True, n_jobs=-1)
    grid_search.fit(X_train.values, y_train.values)

    # make the predictions
    y_pred = grid_search.predict(X_test.values)

    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)

    # confusion matrix on the test data.
    print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    print(pd.DataFrame(confusion_matrix(y_test, y_pred),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))
    return grid_search

In [ ]:
grid_search_clf = grid_search_wrapper(refit_score='recall_score')

In [ ]:
set_option('display.max_row', 200)
set_option('display.width', 1000)
results = pd.DataFrame(grid_search_clf.cv_results_)
results = results.sort_values(by='mean_test_recall_score', ascending=False)
results[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_accuracy_score','param_C', 'param_gamma']].round(3).head(10)